In [1]:
## Import required libraries

## for data
import numpy as np
import pandas as pd

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## for processing
import nltk
import re
import ftfy
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

## WordCloud - Python linrary for creating image wordclouds
from wordcloud import WordCloud


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\otaku7\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\otaku7\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\otaku7\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\otaku7\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
df = pd.read_csv(r"C:\Users\otaku7\Desktop\GP\stress\Stress.csv")

In [5]:
df

,subreddit,post_id,sentence_range,text,label,confidence,social_timestamp
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",1,0.800000,1521614353
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",0,1.000000,1527009817
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,1,0.800000,1535935605
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",1,0.600000,1516429555
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1,0.800000,1539809005
...,...,...,...,...,...,...,...
2833,relationships,7oee1t,"[35, 40]","* Her, a week ago: Precious, how are you? (I i...",0,1.000000,1515187044
2834,ptsd,9p4ung,"[20, 25]",I don't have the ability to cope with it anymo...,1,1.000000,1539827412
2835,anxiety,9nam6l,"(5, 10)",In case this is the first time you're reading ...,0,1.000000,1539269312
2836,almosthomeless,5y53ya,"[5, 10]",Do you find this normal? They have a good rela...,0,0.571429,1488938143


In [6]:
df.drop(columns=['subreddit','post_id','sentence_range','confidence','social_timestamp'],inplace=True)

In [7]:
df

,text,label
0,"He said he had not felt that way before, sugge...",1
1,"Hey there r/assistance, Not sure if this is th...",0
2,My mom then hit me with the newspaper and it s...,1
3,"until i met my new boyfriend, he is amazing, h...",1
4,October is Domestic Violence Awareness Month a...,1
...,...,...
2833,"* Her, a week ago: Precious, how are you? (I i...",0
2834,I don't have the ability to cope with it anymo...,1
2835,In case this is the first time you're reading ...,0
2836,Do you find this normal? They have a good rela...,0


In [38]:
# Expand Contraction
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [52]:
## Function to perform stepwise cleaning process
def tweets_cleaner(tweet):
    
    cleaned_tweets = []
    tweet = tweet.lower() #lowercase
    
  # if url links then don't append to avoid news articles
  # also check tweet length, save those > 5 
    if re.match("(\w+:\/\/\S+)", tweet) == None and len(tweet) > 5:
        
    
    #remove hashtag, @mention, emoji and image URLs
        tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<.>)|(pic\.twitter\.com\/.*)", " ", tweet).split())

        #fix weirdly encoded texts
        tweet = ftfy.fix_text(tweet)

        #expand contraction
        tweet = expandContractions(tweet)


        #remove punctuation
        tweet = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", tweet).split())

        #stop words and lemmatization
        stop_words = set(stopwords.words('english'))
        word_tokens = nltk.word_tokenize(tweet)

        lemmatizer=WordNetLemmatizer()
        filtered_sentence = [lemmatizer.lemmatize(word) for word in word_tokens if not word in stop_words]
        # back to string from list
        tweet = ' '.join(filtered_sentence) # join words with a space in between them

        cleaned_tweets.append(tweet)

    return cleaned_tweets
    

In [41]:
corpus = tweets_cleaner(tweets_arr)

AttributeError: 'list' object has no attribute 'lower'

In [42]:
df['clean_text'] = corpus


ValueError: Length of values (0) does not match length of index (2834)

In [15]:
df

,text,label,clean_text
0,"He said he had not felt that way before, sugge...",1,said felt way suggeted go rest trigger ahead y...
1,"Hey there r/assistance, Not sure if this is th...",0,hey r assistance sure right place post go curr...
2,My mom then hit me with the newspaper and it s...,1,mom hit newspaper shocked would know like play...
3,"until i met my new boyfriend, he is amazing, h...",1,met new boyfriend amazing kind sweet good stud...
4,October is Domestic Violence Awareness Month a...,1,october domestic violence awareness month dome...
...,...,...,...
2833,"* Her, a week ago: Precious, how are you? (I i...",0,week ago precious ignored jan 1 happy year pre...
2834,I don't have the ability to cope with it anymo...,1,ability cope anymore trying lot thing triggeri...
2835,In case this is the first time you're reading ...,0,case first time reading post looking people wi...
2836,Do you find this normal? They have a good rela...,0,find normal good relationship main problem see...


In [17]:

# replace field that's entirely space (or empty) with NaN
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [43]:
df[df['clean_text'].isnull()]

,text,label,clean_text


In [21]:
## Deleting the rows with nan 
df.dropna(subset=['clean_text'], inplace=True)

In [23]:

## Import required libraries

## warnings
import warnings
warnings.filterwarnings("ignore")

## for data
import numpy as np
import pandas as pd

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## TF-IDF 
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import en_core_web_lg



## Train-Test Split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV


## Feature selection
from sklearn import feature_selection

## libraraies for classification
from sklearn.pipeline import Pipeline
import sklearn.metrics as skm
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


## for saving model
import pickle

In [24]:
nlp = en_core_web_lg.load()

In [25]:
all_vectors = pd.np.array([pd.np.array([token.vector for token in nlp(s)]).mean(axis=0) * pd.np.ones((300)) \
                           for s in df['clean_text']])

In [26]:
all_vectors

array([[ 0.23951681,  1.62246275, -0.80913979, ...,  0.36847553,
        -1.91384685,  1.71876025],
       [ 0.08158433,  0.26458135, -1.94797432, ..., -0.54826713,
        -1.81906128,  0.87428159],
       [ 0.02348762,  0.83661175, -2.54537821, ...,  1.50473964,
        -1.38440704,  1.16022635],
       ...,
       [ 0.37162542, -0.19524668, -2.11935306, ..., -2.11180496,
        -2.5505147 ,  2.75154781],
       [ 0.44912869,  1.23822236, -1.86733043, ...,  1.35216367,
        -2.65918159,  0.26423931],
       [ 1.2982229 ,  0.70283377, -1.81676543, ...,  2.22338438,
        -1.70926106,  1.12572825]])

In [27]:
# split out validation dataset for the end
X = all_vectors
Y= df["label"]

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=7)


In [29]:
LR=LogisticRegression()

In [30]:
lr = LR.fit(X_train, Y_train)
train_result = accuracy_score(lr.predict(X_train), Y_train)
    # Test results
y_pred=lr.predict(X_test)
test_result = accuracy_score(y_pred, Y_test)

In [82]:
test_result

0.7320799059929495

In [83]:
data="""
Today has been quite challenging. I woke up feeling overwhelmed by the tasks ahead. The workload at the office seems never-ending, and there's a constant pressure to meet deadlines. I find myself struggling to balance work and personal life, and it's taking a toll on my mental well-being. During lunch, I couldn't shake off the feeling of anxiety and kept replaying all the things I need to do in my mind.

In the afternoon, a meeting with a difficult client added to my stress levels. The pressure to perform well and the fear of making mistakes made my heart race. As the day progressed, I felt the weight of responsibility on my shoulders, and I couldn't stop thinking about how I would manage everything.

In the evening, I tried to unwind by going for a walk, but my mind kept racing with thoughts about the challenges I face. Despite my efforts to relax, I still feel a lingering sense of stress and worry. I hope tomorrow will be a better day
"""

In [84]:
corpus = tweets_cleaner(data)


In [85]:
corpus

['today quite challenging woke feeling overwhelmed task ahead workload office seems never ending constant pressure meet deadline find struggling balance work personal life taking toll mental well lunch could shake feeling anxiety kept replaying thing need mind afternoon meeting difficult client added stress level pressure perform well fear making mistake made heart race day progressed felt weight responsibility shoulder could stop thinking would manage everything evening tried unwind going walk mind kept racing thought challenge face despite effort relax still feel lingering sense stress worry hope tomorrow better day']

In [86]:

## word-embedding
test = pd.np.array([pd.np.array([token.vector for token in nlp(s)]).mean(axis=0) * pd.np.ones((300)) \
                           for s in corpus])

In [87]:

labels_pred = lr.predict(test)
     

In [88]:
labels_pred

array([1], dtype=int64)